In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from IPython.display import display

In [3]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1075,tt1925431,20000000,4636169,Fire with Fire,Bruce Willis|Josh Duhamel|Rosario Dawson|Vince...,David Barrett,Revenge has its own set of rules.,A fireman takes an unexpected course of action...,97,Action|Crime|Drama|Thriller,Industry Entertainment|Grindstone Entertainmen...,8/31/2012,5.7,2012
504,tt0213149,140000000,449220945,Pearl Harbor,Ben Affleck|Josh Hartnett|Kate Beckinsale|Cuba...,Michael Bay,"December 7, 1941 - A day that shall live in in...",The lifelong friendship between Rafe McCawley ...,183,History|Romance|War,Jerry Bruckheimer Films|Touchstone Pictures,5/21/2001,6.6,2001
1738,tt0418279,150000000,709709780,Transformers,Shia LaBeouf|Josh Duhamel|Megan Fox|Rachael Ta...,Michael Bay,Their war. Our world.,"Young teenager, Sam Witwicky becomes involved ...",144,Adventure|Science Fiction|Action,Paramount Pictures|DreamWorks SKG|Amblin Enter...,6/27/2007,6.6,2007
1746,tt0414055,25000000,74237563,Elizabeth: The Golden Age,Cate Blanchett|Clive Owen|Geoffrey Rush|Lauren...,Shekhar Kapur,Woman. Warrior. Queen.,This movie is a Sequel to the First Elizabeth ...,114,Drama|History|Romance,Universal Pictures|StudioCanal|Working Title F...,9/9/2007,6.4,2007
821,tt0376136,45000000,24000000,The Rum Diary,Johnny Depp|Amber Heard|Aaron Eckhart|Michael ...,Bruce Robinson,One part outrage. One part justice. Three part...,Tired of the noise and madness of New York and...,120,Drama|Comedy,FilmEngine|Infinitum Nihil|GK Films,10/13/2011,5.5,2011


In [4]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [106]:
#=============================================================================
answers = {} # создадим словарь для ответов

# тут другие ваши предобработки колонок например:

#the time given in the dataset is in string format.
#So we need to change this in datetime format
# ...

# Для 6 задания
# Добавляем стоблец Прибыль - profit
data = pd.read_csv('movie_bd_v5.csv')
column_profit = pd.concat([pd.DataFrame(
                                [data.revenue.values[k]
                                -data.budget.values[k]],
                                columns=['profit'])
                            for k in range(data.budget.count())],
                            ignore_index=True)
data = pd.concat([data,column_profit],axis=1)

# Добавляем стоблец Количество символов в назнвании фильма - str_title
column_str_title = pd.concat([pd.DataFrame(
                                [len(data.original_title.values[k])],
                                columns=['str_title'])
                            for k in range(data.original_title.count())],
                            ignore_index=True)
data = pd.concat([data,column_str_title],axis=1)

# Добавляем стоблец количество символов в описании фильма - str_overview
column_str_overview = pd.concat([pd.DataFrame(
                                [len(data.overview.values[k].split(' '))],
                                columns=['str_overview'])
                            for k in range(data.overview.count())],
                            ignore_index=True)
data = pd.concat([data,column_str_overview],axis=1)

# К заданию 21
dict_month = {
    '1' : 'Январь',
    '2' : 'Февраль',
    '3' : 'Март',
    '4' : 'Апрель',
    '5' : 'Май',
    '6' : 'Июнь',
    '7' : 'Июль',
    '8' : 'Август',
    '9' : 'Сентябрь',
    '10' : 'Октябрь',
    '11' : 'Ноябрь',
    '12' : 'Декабрь'
}

def str_sum(data):
    '''
        Возвращает DataFrame со списком всех компаний
        и количеством символов в их названиях
    '''
    str_dict = {}

    for k in range(data.production_companies.values.size):

        str_list = data.production_companies.values[k].split('|')

        for j in range(len(str_list)):
            
            if not str_list[j] in str_dict:
                str_dict[str_list[j]] = data.str_title.values[k]
            elif str_dict[str_list[j]] < data.str_title.values[k]:
                str_dict[str_list[j]] = data.str_title.values[k]

    sum_values = list(str_dict.values());
    sum_frame = pd.DataFrame([[sum_values[g]]
                                    for g in range(len(sum_values))],
                                columns=['values'],
                                index=list(str_dict.keys()))
    return sum_frame;

# Функция к 11, 13, 16 заданию и не только
def dict_split_item(data, key, sep='|',split=True,summ=False,
                        keySum=False,indexSplit=-1):
    '''
        Функция разделяющая строку по разделителю и составляющая словарь
        значений которых больше преобразующая его значения в DataFrame
        и вовращает рузультат DataFrame
    '''
    dict_value = {}

    for k in range(data[key].values.size):

        if split:
            list_item = data[key].values[k].split(sep)

            if indexSplit >= 0:
                if summ and keySum:
                    if not list_item[indexSplit] in dict_value:
                        dict_value[
                            list_item[indexSplit]] = data[keySum].values[k]
                    else:
                        dict_value[
                            list_item[indexSplit]] += data[keySum].values[k]
                else:
                    if not list_item[indexSplit] in dict_value:
                        dict_value[list_item[indexSplit]] = 1
                    else:
                        dict_value[list_item[indexSplit]] += 1
            else:

                for j in range(len(list_item)):

                    if summ and keySum:
                        if not list_item[j] in dict_value:
                            dict_value[
                                list_item[j]] = data[keySum].values[k]
                        else:
                            dict_value[
                                list_item[j]] += data[keySum].values[k]
                    else:
                        if not list_item[j] in dict_value:
                            dict_value[list_item[j]] = 1
                        else:
                            dict_value[list_item[j]] += 1
        else:
            list_item = data[key].values[k]

            if summ and keySum:
                if not list_item in dict_value:
                    dict_value[list_item] = data[keySum].values[k]
                else:
                    dict_value[list_item] += data[keySum].values[k]
            else:
                if not list_item in dict_value:
                        dict_value[list_item] = 1
                else:
                    dict_value[list_item] += 1

    list_value = list(dict_value.values())
    frame_value = pd.DataFrame([[list_value[g]]
                                for g in range(len(list_value))],
                    columns=['values'],
                    index=list(dict_value.keys()))

    return frame_value

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [6]:
#=============================================================================
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа
# +
answers['1'] = '723. Pirates of the Caribbean: On Stranger Tides (tt1298650)'
# если ответили верно, можете добавить комментарий со значком "+"

In [7]:
#=============================================================================
# тут пишем ваш код для решения данного вопроса:
index = data[data.budget == data.budget.max()].index[0]
original_title = data[data.budget
                    == data.budget.max()].original_title.values[0]
imdb_id = data[data.budget== data.budget.max()].imdb_id.values[0]
budget = data.budget.max()

print(f'{index}. {original_title} ({imdb_id}) ${budget}')

723. Pirates of the Caribbean: On Stranger Tides (tt1298650) $380000000


ВАРИАНТ 2

In [8]:
# можно добавлять разные варианты решения

# 2. Какой из фильмов самый длительный (в минутах)?

In [9]:
# думаю логику работы с этим словарем вы уже поняли, 
# по этому не буду больше его дублировать
# +
answers['2'] = '1157. Gods and Generals (tt0279111)'

In [10]:
#=============================================================================
index = data[data.runtime == data.runtime.max()].index[0]
original_title = data[data.runtime
                        == data.runtime.max()].original_title.values[0]
imdb_id = data[data.runtime == data.runtime.max()].imdb_id.values[0]
runtime = data.runtime.max()

print(f'{index}. {original_title} ({imdb_id}) {runtime} min')

1157. Gods and Generals (tt0279111) 214 min


# 3. Какой из фильмов самый короткий (в минутах)?





In [11]:
# +
answers['3'] = '768. Winnie the Pooh (tt1449283)'

In [12]:
#=============================================================================
index = data[data.runtime == data.runtime.min()].index[0]
original_title = data[data.runtime
                        == data.runtime.min()].original_title.values[0]
imdb_id = data[data.runtime == data.runtime.min()].imdb_id.values[0]
runtime = data.runtime.min()

print(f'{index}. {original_title} ({imdb_id}) {runtime} min')

768. Winnie the Pooh (tt1449283) 63 min


# 4. Какова средняя длительность фильмов?


In [13]:
# +
answers['4'] = '110'

In [14]:
round(data.runtime.mean(),0)

110.0

# 5. Каково медианное значение длительности фильмов? 

In [15]:
# +
answers['5'] = '107'

In [16]:
round(data.runtime.median())

107

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [17]:
# +
answers['6'] = 'Avatar (tt0499549)'

In [18]:
# лучше код получения столбца profit вынести в Предобработку что в начале
#=============================================================================
index_profit = data[data.profit == data.profit.max()].index[0]
original_title_profit = data[data.profit
                                == data.profit.max()].original_title.values[0]
imdb_id_profit = data[data.profit == data.profit.max()].imdb_id.values[0]
max_profit = data.profit.max()

print(f'{index_profit}. {original_title_profit}',
        f'({imdb_id_profit}) {max_profit}')

239. Avatar (tt0499549) 2544505847


# 7. Какой фильм самый убыточный? 

In [19]:
# +
answers['7'] = 'The Lone Ranger (tt1210819)'

In [20]:
#=============================================================================
index_profit_min = data[data.profit == data.profit.min()].index[0]
original_title_profit_min = data[data.profit
                                == data.profit.min()].original_title.values[0]
imdb_id_profit_min = data[data.profit == data.profit.min()].imdb_id.values[0]
min_profit = data.profit.min()

print(f'{index_profit_min}. {original_title_profit_min}',
        f'({imdb_id_profit_min}) {min_profit}')

1245. The Lone Ranger (tt1210819) -165710090


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [21]:
# +
answers['8'] = '1478'

In [22]:
#=============================================================================
revenue_budget = data[data.revenue > data.budget]
display(revenue_budget.revenue.count())

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [23]:
# +
answers['9'] = 'The Dark Knight (tt0468569)'

In [24]:
#=============================================================================
revenue = data[data.release_year == 2008]
revenue = revenue[revenue.revenue == revenue.revenue.max()]
index_revenue = revenue.index[0]
original_title_revenue = revenue.original_title.values[0]
imdb_id_revenue = revenue.imdb_id.values[0]
max_revenue = data.revenue.max()

print(f'{index_revenue}. {original_title_revenue} ({imdb_id_revenue})',
            f"{max_revenue}")

599. The Dark Knight (tt0468569) 2781505847


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [25]:
# +
answers['10'] = 'The Lone Ranger (tt1210819)'

In [26]:
#=============================================================================
profit = data[(data.profit == data.profit.min())
                & (data.release_year >= 2012)
                & (data.release_year <= 2014)]
index_min_year = profit.index[0]
original_title_min_year = profit.original_title.values[0]
imdb_id_min_year = profit.imdb_id.values[0]
min_year = profit.profit.min()

print(f'{index_min_year}. {original_title_min_year} ({imdb_id_min_year})',
        f'{min_year}')

1245. The Lone Ranger (tt1210819) -165710090


# 11. Какого жанра фильмов больше всего?

In [27]:
# +
answers['11'] = 'Drama'

In [28]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале
#=============================================================================
genres_frame = dict_split_item(data,'genres')
genres = genres_frame[genres_frame.values == genres_frame.values.max()]
genres_index_max = genres.index[0]

print(f'{genres_index_max} {genres_frame.values.max()}')

Drama 782


ВАРИАНТ 2

In [29]:
#=============================================================================

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [30]:
# +
answers['12'] = 'Drama'

In [31]:
#=============================================================================
genres_max = dict_split_item(data[data.profit > 0], 'genres')
index_max_value = genres_max[genres_max.values == genres_max.values.max()]

print(f'{index_max_value.index[0]} {genres_max.values.max()}')

Drama 560


# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [32]:
# +
answers['13'] = 'Peter Jackson'

In [33]:
#=============================================================================
director_frame = dict_split_item(data, 'director',summ=True,keySum='revenue')
director_index = director_frame[director_frame.values
                                    == director_frame.values.max()].index[0]

display(f'{director_index} {director_frame.values.max()}')

'Peter Jackson 6490593685'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [34]:
# +
answers['14'] = 'Robert Rodriguez'

In [35]:
#=============================================================================
data_action = data[data.genres.str.contains('Action',na=False)]

director_frame = False
director_frame = dict_split_item(data_action,'director')
director_name = director_frame[director_frame.values
                                        == director_frame.values.max()]
director_value = director_frame[director_frame.values
                                    == director_frame.values.max()]
display(f'{director_value.index[0]} {director_value.values[0]}')


'Robert Rodriguez [9]'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [36]:
# +
answers['15'] = 'Chris Hemsworth'

In [37]:
#=============================================================================
data_2012 = data[data.release_year == 2012]

cast_revenue = {}
for c in range(data_2012.cast.values.size):

    cast_revenue_list = data_2012.cast.values[c].split('|');

    for j in range(len(cast_revenue_list)):
        if not cast_revenue_list[j] in cast_revenue:
            cast_revenue[
                cast_revenue_list[j]] = data_2012.revenue.values[c]
        else:
            cast_revenue[
                cast_revenue_list[j]] += data_2012.revenue.values[c]

cast_revenue_value = list(cast_revenue.values());
cast_revenue_frame = pd.DataFrame([[cast_revenue_value[g]]
                                    for g in range(len(cast_revenue_value))],
                        columns=['values'],
                        index=list(cast_revenue.keys()))

result_cast = cast_revenue_frame[cast_revenue_frame.values
                                    == cast_revenue_frame.values.max()]
display(f'{result_cast.index[0]} {cast_revenue_frame.values.max()}')


'Chris Hemsworth 2027450773'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [38]:
# +
answers['16'] = 'Matt Damon'

In [39]:
#=============================================================================
data_budget_cast = data[data.budget > data.budget.mean()]

cast_budget_frame = dict_split_item(data_budget_cast, 'cast')
index_budget = cast_budget_frame[cast_budget_frame.values
                                    == cast_budget_frame.values.max()]

display(f'{index_budget.index[0]} {cast_budget_frame.values.max()}')




'Matt Damon 18'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [40]:
# +
answers['17'] = 'Action'

In [41]:
#=============================================================================
data_cast_cage = data[data.cast.str.contains('Nicolas Cage',na=False)]
film_genres_cast = dict_split_item(data_cast_cage,'genres')
index_cage = film_genres_cast[film_genres_cast.values
                                == film_genres_cast.values.max()]

display(f'{index_cage.index[0]} {film_genres_cast.values.max()}')

'Action 17'

# 18. Самый убыточный фильм от Paramount Pictures

In [42]:
# +
answers['18'] = 'K-19: The Widowmaker (tt0267626)'

In [43]:
#=============================================================================
data_company = data[data.production_companies.str.contains(
                                                    'Paramount Pictures',
                                                    na=False)]
company_profit = data_company[data_company.profit
                                == data_company.profit.min()]
company_imdb_id = company_profit.imdb_id.values[0]
company_original_title = company_profit.original_title.values[0]
display(f'{company_original_title} ({company_imdb_id})')

'K-19: The Widowmaker (tt0267626)'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [44]:
# +
answers['19'] = '2015'

In [45]:
#=============================================================================
data_revenue = dict_split_item(
                    data,'release_year',
                    split=False,summ=True,keySum='revenue')
year_revenue = data_revenue[data_revenue.values
                            == data_revenue.values.max()]
display(f'{year_revenue.index[0]} {year_revenue.values[0]}')

'2015 [25449202382]'

# 20. Какой самый прибыльный год для студии Warner Bros?

In [46]:
# +
answers['20'] = '2014'

In [47]:
#=============================================================================
data_company_w = data[data.production_companies.str.contains(
                                                    'Warner Bros',
                                                    na=False)]
data_year_company = dict_split_item(data_company_w,
                                    'release_year',
                                    split=False,
                                    summ=True,
                                    keySum='profit')
year_company_profit = data_year_company[data_year_company.values
                                        == data_year_company.values.max()]
display(f'{year_company_profit.index[0]} {year_company_profit.values[0]}')

'2014 [2295464519]'

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [48]:
# +
answers['21'] = 'Сентябрь'

In [50]:
#=============================================================================
month_years_sum = dict_split_item(data, 'release_date',
                                    sep='/',split=True,summ=False,
                                    keySum=False,indexSplit=0)
year_month_max = month_years_sum[month_years_sum.values
                                    == month_years_sum.values.max()]

display(f'{dict_month[year_month_max.index[0]]} {year_month_max.values[0]}')

'Сентябрь [227]'

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [51]:
# +
answers['22'] = '450'

In [52]:
#=============================================================================
films_summer_sum = dict_split_item(data, 'release_date',
                                    sep='/',split=True,summ=False,
                                    keySum=False,indexSplit=0)
films_summer = month_years_sum[(films_summer_sum.index == '6')
                                | (films_summer_sum.index == '7')
                                | (films_summer_sum.index == '8')
                            ].sum().values[0]
display(films_summer)

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [53]:
# +
answers['23'] = 'Peter Jackson'

In [87]:
#=============================================================================
films_data = data[data.release_date.str.match('12/',na=False)
                    | data.release_date.str.match('1/',na=False)
                    | data.release_date.str.match('2/',na=False)]

director_winter = dict_split_item(films_data,'director')
director_max = director_winter[director_winter.values
                                == director_winter.values.max()]

display(f'{director_max.index[0]} {director_max.values[0]}')


'Peter Jackson [7]'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [102]:
# +
answers['24'] = 'Warner Bros.'

In [103]:
#=============================================================================
#films_com = str_sum(data)#data.production_companies.unique()
studio_str_frame = dict_split_item(data, 'production_companies', summ=True,
                                    keySum='str_title')
studio_str = studio_str_frame[studio_str_frame.values
                                == studio_str_frame.values.max()]
display(f'{studio_str.index[0]} {studio_str.values[0]}')
#studio_str = films_com[films_com.values == films_com.values.max()]
#display(f'{studio_str.index[0]} {studio_str.values[0]}')

'Warner Bros. [2616]'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [57]:
# +
answers['25'] = 'Universal Pictures'

In [107]:
#=============================================================================
studio_overview_frame = dict_split_item(data, 'production_companies',
                                        summ=True, keySum='str_overview')
studio_overview = studio_overview_frame[studio_overview_frame.values
                                == studio_overview_frame.values.max()]
display(f'{studio_overview.index[0]} {studio_overview.values[0]}')

#display(data.str_overview)

'Universal Pictures [9287]'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [58]:
# +
answers['26'] = ''

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [59]:
# +
answers['27'] = 'Daniel Radcliffe & Rupert Grint'

ВАРИАНТ 2

# Submission

In [60]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'22': '450', '23': '', '24': '', '25': '', '26': '', '27': ''}

In [61]:
# и убедиться что ни чего не пропустил)
len(answers)

6